In [1]:
import numpy as np
import pandas as pd
# Just to plot the time series and see what it looks like
#import matplotlib.pyplot as plt

%load_ext rpy2.ipython

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import os

def load_smooth_dev():
    """Load the smooth package in development mode"""
    # Path to your smooth package
    smooth_path = "~/Cursor/smooth/"  # Adjust this path if needed
    
    # Load devtools and then load_all() the smooth package
    ro.r(f'''
    if (!requireNamespace("devtools", quietly=TRUE)) {{
        install.packages("devtools", repos="https://cran.rstudio.com/")
    }}
    devtools::load_all("{smooth_path}")
    ''')
    
    print("Smooth package loaded in development mode")

# Load the package in development mode
load_smooth_dev()

ℹ Loading smooth


R[write to console]: Loading required package: greybox

R[write to console]: Package "greybox", v2.0.4.41003 loaded.


R[write to console]: This is package "smooth", v4.1.2.41005


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages



Smooth package loaded in development mode


In [2]:
#### Global level and ETS(A,N,N) ####
np.random.seed(33)  # For reproducibility
n_points = 100
time_series = np.random.normal(100, 10, n_points)  # Random errors
#dates = pd.date_range(start='2023-01-01', periods=n_points, freq='ME')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series})

In [3]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12))

forecast(model, h = 12)

Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61957 99.61957 99.61957 99.61957 99.61957 99.61957 99.61957 99.61957
 [9] 99.61957 99.61957 99.61957 99.61957


In [4]:
#### Local level and ETS(A,N,N) ####
np.random.seed(41)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + errors[i+1]
#dates = pd.date_range(start='2023-01-01', periods=n_points, freq='ME')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series})

In [5]:
%%R -i ts_df

# ETS(A,N,N) with alpha=0.1
model <- adam(ts_df, model = "ANN", lags = c(12), persistence = 0.1)

cat("ETS(A,N,N) with alpha=0.1:\n")
forecast(model, h = 12) |> print()

# ETS(A,N,N) with estimated alpha
model <- adam(ts_df, model = "ANN", lags = c(12))
cat("\nETS(A,N,N) with estimated parameters:\n")
print(model$B)
forecast(model, h = 12) |> print()

# ETS(M,N,N) with estimated alpha
model <- adam(ts_df, model = "MNN", lags = c(12), distribution="dnorm")
cat("\nETS(M,N,N) with estimated parameters:\n")
forecast(model, h = 12) |> print()


ETS(A,N,N) with alpha=0.1:
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 83.22599 83.22599 83.22599 83.22599 83.22599 83.22599 83.22599 83.22599
 [9] 83.22599 83.22599 83.22599 83.22599

ETS(A,N,N) with estimated parameters:
      alpha       level 
 0.06531787 96.99069808 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 84.70888 84.70888 84.70888 84.70888 84.70888 84.70888 84.70888 84.70888
 [9] 84.70888 84.70888 84.70888 84.70888

ETS(M,N,N) with estimated parameters:
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 84.83866 84.83866 84.83866 84.83866 84.83866 84.83866 84.83866 84.83866
 [9] 84.83866 84.83866 84.83866 84.83866


In [6]:
#### Local trend and ETS(A,A,N) ####
np.random.seed(42)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
trend = np.random.normal(0.5, 2, n_points)  # Random trend
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
#dates = pd.date_range(start='2023-01-01', periods=n_points, freq='ME')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series})


In [7]:
%%R -i ts_df

# ETS(A,A,N)
model1 <- adam(ts_df, model = "AAN", lags = c(12), persistence = c(0.1, 0.02), distribution="dnorm")

cat("ETS(A,A,N):\n")
forecast(model1, h = 12) |> print()

# ETS(A,Ad,N) estimated
cat("\nETS(A,Ad,N) estimated parameters:\n")
print(adam(ts_df, model = "AAdN", lags = c(12), distribution="dnorm")$B)

# ETS(A,Ad,N)
model2 <- adam(ts_df, model = "AAdN", lags = c(12), persistence = c(0.1, 0.02), distribution="dnorm")

cat("\nETS(A,Ad,N):\n")
forecast(model2, h = 12) |> print()

ETS(A,A,N):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.2555 165.7640 166.2724 166.7808 167.2892 167.7977 168.3061 168.8145
 [9] 169.3230 169.8314 170.3398 170.8483

ETS(A,Ad,N) estimated parameters:
       alpha         beta          phi        level        trend 
  0.16827523   0.01042486   0.98630131 104.77485790  -0.38092112 

ETS(A,Ad,N):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 163.2357 163.7013 164.1408 164.5555 164.9469 165.3163 165.6649 165.9939
 [9] 166.3044 166.5974 166.8740 167.1350


In [8]:
%%R -i ts_df

# ETS(M,A,N)
model1 <- adam(ts_df, model = "MAN", lags = c(12), persistence = c(0.1, 0.02), distribution="dnorm")

cat("ETS(M,A,N):\n")
forecast(model1, h = 12) |> print()

# ETS(M,Ad,N) estimated
cat("\nETS(M,Ad,N) estimated parameters:\n")
print(adam(ts_df, model = "MAdN", lags = c(12), distribution="dnorm")$B)

# ETS(M,Ad,N)
model2 <- adam(ts_df, model = "MAdN", lags = c(12), persistence = c(0.1, 0.02), distribution="dnorm")

cat("\nETS(M,Ad,N):\n")
forecast(model2, h = 12) |> print()

ETS(M,A,N):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.2563 165.7645 166.2728 166.7811 167.2894 167.7977 168.3060 168.8143
 [9] 169.3226 169.8309 170.3392 170.8475

ETS(M,Ad,N) estimated parameters:
       alpha         beta          phi        level        trend 
  0.14969093   0.01327518   0.96754121 105.75620898 

 -0.46234686 

ETS(M,Ad,N):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 163.1952 163.6595 164.0968 164.5088 164.8968 165.2622 165.6065 165.9307
 [9] 166.2361 166.5238 166.7948 167.0500


In [9]:
%%R -i ts_df

# ETS(M,M,N)
model1 <- adam(ts_df, model = "MMN", lags = c(12), persistence = c(0.1, 0.02), distribution="dnorm")

cat("ETS(M,M,N):\n")
print(model1$B)
forecast(model1, h = 12) |> print()

# ETS(M,Md,N) estimated
cat("\nETS(M,Md,N) estimated parameters:\n")
print(adam(ts_df, model = "MMdN", lags = c(12), distribution="dnorm")$B)

# ETS(M,Md,N)
model2 <- adam(ts_df, model = "MMdN", lags = c(12), persistence = c(0.1, 0.02), distribution="dnorm")

cat("\nETS(M,Md,N):\n")
forecast(model2, h = 12) |> print()


ETS(M,M,N):
      level       trend 
107.9006421   0.9932798 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.5615 166.0806 166.7931 167.0704 167.6807 167.8501 168.5623 169.2153
 [9] 169.7927 169.9725 170.8213 171.0304

ETS(M,Md,N) estimated parameters:
       alpha         beta          phi        level        trend 
  0.14927185   0.01276844   0.96512552 106.24511408   0.99496933 

ETS(M,Md,N):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 163.1433 163.5274 164.0098 164.5318 164.5706 165.0050 165.5159 165.6156
 [9] 166.4636 166.2174 166.6896 166.9163


In addition: Warning messages:
1: Your model has a potentially explosive multiplicative trend. I cannot do anything about it, so please just be careful. 
2: Your model has a potentially explosive multiplicative trend. I cannot do anything about it, so please just be careful. 


In [10]:
#### Seasonal data ####
np.random.seed(42)  # For reproducibility
n_points = 120
errors = (1+np.random.normal(0, 0.1, n_points))  # Random errors
trend = np.random.normal(0.5, 2, n_points)  # Random trend
seasonal_sd = 0.2
seasonal_pattern = np.exp(np.random.normal(0, seasonal_sd, 12))  # Random seasonal pattern
seasonal_pattern = seasonal_pattern / np.mean(seasonal_pattern)  # Normalize to sum to 1
time_series = np.zeros(n_points)
time_series[0] = 200 * seasonal_pattern[0] * errors[0]
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = ((time_series[i] / seasonal_pattern[(i) % 12]-trend[i]) * errors[i] ** (0.1-1) + trend[i+1]) * seasonal_pattern[(i+1) % 12] * errors[i+1]
ts_df = pd.DataFrame({'value': time_series})


In [17]:
%%R -i ts_df

# ETS(A,A,A)
model1 <- adam(ts_df, model = "AAA", lags = c(12), distribution="dnorm")

cat("ETS(A,A,A):\n")
print(model1$B)
forecast(model1, h = 12) |> print()

# ETS(A,Ad,A) estimated
model2 <- adam(ts_df, model = "AAdA", lags = c(12), distribution="dnorm")

cat("\nETS(A,Ad,A):\n")
forecast(model2, h = 12) |> print()

ETS(A,A,A):
        alpha          beta         gamma         level         trend 
 2.161811e-02  1.579075e-02  5.859355e-04  2.218595e+02 -7.471062e-01 
   seasonal_1    seasonal_2    seasonal_3    seasonal_4    seasonal_5 
-2.789793e+01 -1.110634e+01 -1.877190e-01  3.397825e+01 -4.309394e+01 
   seasonal_6    seasonal_7    seasonal_8    seasonal_9   seasonal_10 
 1.168707e+00 -1.501950e+01 -2.036129e+01  7.238855e+01  9.665323e+00 
  seasonal_11 
-3.498305e+01 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 139.4253 157.0797 168.9017 203.9372 127.7433 172.8483 157.5823 153.1201
 [9] 246.7386 184.9105 141.1466 212.4595

ETS(A,Ad,A):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 143.3686 159.1775 170.6195 206.3118 128.7002 165.9054 158.7444 152.9272
 [9] 245.0987 184.7617 139.8602 212.0961


In [18]:
%%R -i ts_df

# ETS(M,A,M)
model3 <- adam(ts_df, model = "MAM", lags = c(12), distribution="dnorm")

cat("ETS(M,A,M):\n")
print(model3$B)
forecast(model3, h = 12) |> print()

# ETS(M,Ad,M) estimated
model4 <- adam(ts_df, model = "MAdM", lags = c(12), distribution="dnorm")

cat("\nETS(M,Ad,M):\n")
forecast(model4, h = 12) |> print()


ETS(M,A,M):
       alpha         beta        gamma        level        trend   seasonal_1 
1.481591e-01 1.426804e-02 7.384620e-04 2.037820e+02 6.272065e-01 8.650025e-01 
  seasonal_2   seasonal_3   seasonal_4   seasonal_5   seasonal_6   seasonal_7 
9.461651e-01 1.032165e+00 1.201518e+00 7.706866e-01 9.885534e-01 9.266625e-01 
  seasonal_8   seasonal_9  seasonal_10  seasonal_11 
9.073598e-01 1.417307e+00 1.067171e+00 8.234215e-01 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 146.5487 160.5128 175.3330 204.3908 131.2754 168.6074 158.2698 155.1663
 [9] 242.7110 183.0018 141.3871 212.3149

ETS(M,Ad,M):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 145.2918 161.0047 173.9070 202.6789 129.8380 167.2848 157.8726 154.0776
 [9] 241.7977 181.0301 139.8114 210.1557


In [19]:
%%R -i ts_df
# ETS(M,M,M)
model5 <- adam(ts_df, model = "MMM", lags = c(12), distribution="dnorm")

cat("ETS(M,M,M):\n")
print(model5$B)
forecast(model5, h = 12) |> print()

# ETS(M,Md,M) estimated
model6 <- adam(ts_df, model = "MMdM", lags = c(12), distribution="dnorm")

cat("\nETS(M,Md,M):\n")
forecast(model6, h = 12) |> print()

ETS(M,M,M):
       alpha         beta        gamma        level        trend   seasonal_1 
1.421328e-01 3.263737e-02 2.000175e-03 2.889158e+02 9.775630e-01 8.632702e-01 
  seasonal_2   seasonal_3   seasonal_4   seasonal_5   seasonal_6   seasonal_7 
9.393712e-01 1.035164e+00 1.198405e+00 7.638555e-01 9.888694e-01 9.310523e-01 
  seasonal_8   seasonal_9  seasonal_10  seasonal_11 
9.037185e-01 1.435445e+00 1.068880e+00 8.222703e-01 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 148.8959 162.7222 180.2983 209.6383 134.3044 174.5189 165.2093 160.6445
 [9] 257.0602 192.4393 148.8488 225.6364

ETS(M,Md,M):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 147.2981 162.3072 171.8453 202.9607 128.7773 164.2917 156.7665 153.4287
 [9] 241.5124 181.5702 137.2505 208.6129


In addition: Warning messages:
1: Your model has a potentially explosive multiplicative trend. I cannot do anything about it, so please just be careful. 
2: Your model has a potentially explosive multiplicative trend. I cannot do anything about it, so please just be careful. 
